In [1]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, AveragePooling2D, Dropout
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential
from os import getcwd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

In [2]:
train_dir = 'C:/Users/Asus/COVID19 datasets - Limited/Training'
validation_dir = 'C:/Users/Asus/COVID19 datasets - Limited/Testing'

train_bacterial_dir = train_dir+'/BACTERIAL'+'/train'
train_covid_dir = train_dir+'/COVID19'+'/train'
train_normal_dir = train_dir+'/NORMAL'+'/train'
train_viral_dir = train_dir+'/VIRAL'+'/train'

validation_bacterial_dir = validation_dir+'/BACTERIAL'+'/test'
validation_covid_dir = validation_dir+'/COVID19'+'/test'
validation_normal_dir = validation_dir+'/NORMAL'+'/test'
validation_viral_dir = validation_dir+'/VIRAL'+'/test'

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                   batch_size = 16,
                                                   class_mode = 'categorical',
                                                   target_size = (224, 224),
                                                   shuffle = True)     

validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                        batch_size = 16,
                                                        class_mode = 'categorical',
                                                        target_size = (224, 224),
                                                        shuffle = True)

Found 1000 images belonging to 4 classes.
Found 200 images belonging to 4 classes.


In [4]:
local_weights_file = f"{getcwd()}/xception_weights_tf_dim_ordering_tf_kernels_notop.h5"

pre_trained_model = Xception(include_top = False, 
                             weights = None,
                             input_shape = (224, 224, 3),
                             pooling = 'avg')

pre_trained_model.load_weights(local_weights_file)

pre_trained_model.summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                          

 block4_sepconv1_bn (BatchNorma  (None, 28, 28, 728)  2912       ['block4_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block4_sepconv2_act (Activatio  (None, 28, 28, 728)  0          ['block4_sepconv1_bn[0][0]']     
 n)                                                                                               
                                                                                                  
 block4_sepconv2 (SeparableConv  (None, 28, 28, 728)  536536     ['block4_sepconv2_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block4_sepconv2_bn (BatchNorma  (None, 28, 28, 728)  2912       ['block4_sepconv2[0][0]']        
 lization)

 n)                                                                                               
                                                                                                  
 block7_sepconv1 (SeparableConv  (None, 14, 14, 728)  536536     ['block7_sepconv1_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block7_sepconv1_bn (BatchNorma  (None, 14, 14, 728)  2912       ['block7_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block7_sepconv2_act (Activatio  (None, 14, 14, 728)  0          ['block7_sepconv1_bn[0][0]']     
 n)                                                                                               
          

 block9_sepconv3_bn (BatchNorma  (None, 14, 14, 728)  2912       ['block9_sepconv3[0][0]']        
 lization)                                                                                        
                                                                                                  
 add_7 (Add)                    (None, 14, 14, 728)  0           ['block9_sepconv3_bn[0][0]',     
                                                                  'add_6[0][0]']                  
                                                                                                  
 block10_sepconv1_act (Activati  (None, 14, 14, 728)  0          ['add_7[0][0]']                  
 on)                                                                                              
                                                                                                  
 block10_sepconv1 (SeparableCon  (None, 14, 14, 728)  536536     ['block10_sepconv1_act[0][0]']   
 v2D)     

                                                                                                  
 block12_sepconv3_act (Activati  (None, 14, 14, 728)  0          ['block12_sepconv2_bn[0][0]']    
 on)                                                                                              
                                                                                                  
 block12_sepconv3 (SeparableCon  (None, 14, 14, 728)  536536     ['block12_sepconv3_act[0][0]']   
 v2D)                                                                                             
                                                                                                  
 block12_sepconv3_bn (BatchNorm  (None, 14, 14, 728)  2912       ['block12_sepconv3[0][0]']       
 alization)                                                                                       
                                                                                                  
 add_10 (A

In [5]:
for layer in pre_trained_model.layers[:40]:
    layer.trainable = False

In [6]:
last_layer = pre_trained_model.get_layer('add_7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 14, 14, 728)


In [7]:
x = AveragePooling2D((4, 4)) (last_output)
x = Flatten()(x)
x = Dense(64, activation = 'relu')(x)
x = Dropout(0.5)(x)                  
x = Dense(4, activation = 'softmax')(x)           

model = Model(pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(learning_rate = 0.001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

 block4_sepconv1_bn (BatchNorma  (None, 28, 28, 728)  2912       ['block4_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block4_sepconv2_act (Activatio  (None, 28, 28, 728)  0          ['block4_sepconv1_bn[0][0]']     
 n)                                                                                               
                                                                                                  
 block4_sepconv2 (SeparableConv  (None, 28, 28, 728)  536536     ['block4_sepconv2_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block4_sepconv2_bn (BatchNorma  (None, 28, 28, 728)  2912       ['block4_sepconv2[0][0]']        
 lization)

 n)                                                                                               
                                                                                                  
 block7_sepconv1 (SeparableConv  (None, 14, 14, 728)  536536     ['block7_sepconv1_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block7_sepconv1_bn (BatchNorma  (None, 14, 14, 728)  2912       ['block7_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block7_sepconv2_act (Activatio  (None, 14, 14, 728)  0          ['block7_sepconv1_bn[0][0]']     
 n)                                                                                               
          

 block9_sepconv3_bn (BatchNorma  (None, 14, 14, 728)  2912       ['block9_sepconv3[0][0]']        
 lization)                                                                                        
                                                                                                  
 add_7 (Add)                    (None, 14, 14, 728)  0           ['block9_sepconv3_bn[0][0]',     
                                                                  'add_6[0][0]']                  
                                                                                                  
 average_pooling2d (AveragePool  (None, 3, 3, 728)   0           ['add_7[0][0]']                  
 ing2D)                                                                                           
                                                                                                  
 flatten (Flatten)              (None, 6552)         0           ['average_pooling2d[0][0]']      
          

In [8]:
history = model.fit(train_generator,
                    verbose = 1,
                    batch_size = 16,
                    epochs = 15)

Epoch 1/15
63/63 [==============================] - 229s 3s/step - loss: 2.3249 - accuracy: 0.4860
Epoch 2/15
63/63 [==============================] - 209s 3s/step - loss: 1.1314 - accuracy: 0.5580
Epoch 3/15
63/63 [==============================] - 208s 3s/step - loss: 1.0883 - accuracy: 0.5420
Epoch 4/15
63/63 [==============================] - 207s 3s/step - loss: 0.9734 - accuracy: 0.5760
Epoch 5/15
63/63 [==============================] - 208s 3s/step - loss: 0.9529 - accuracy: 0.6230
Epoch 6/15
63/63 [==============================] - 207s 3s/step - loss: 0.9035 - accuracy: 0.6340
Epoch 7/15
63/63 [==============================] - 207s 3s/step - loss: 0.7598 - accuracy: 0.6870
Epoch 8/15
63/63 [==============================] - 208s 3s/step - loss: 0.6885 - accuracy: 0.7220
Epoch 9/15
63/63 [==============================] - 205s 3s/step - loss: 0.6712 - accuracy: 0.7120
Epoch 10/15
63/63 [==============================] - 203s 3s/step - loss: 0.6460 - accuracy: 0.7340
Epoch 11/

In [9]:
results = model.evaluate(validation_generator, batch_size=128)
print("test loss, test acc:", results)

13/13 [==============================] - 11s 790ms/step - loss: 1.3323 - accuracy: 0.8350
test loss, test acc: [1.332303762435913, 0.8349999785423279]
